In [41]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib 
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import seaborn as sns
from scipy.stats import skew
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import datetime
from scipy.stats import skew
from scipy.stats import norm
import os
import  plotly.plotly as py
import plotly.graph_objs as go
import pickle
from src import utils


#Environment settings
data_path_in = 'Data/input/'
data_path_out= 'Data/ouput/'

In [42]:
# Deserialize previously saved data from "preprocessing"
train = pd.read_csv(data_path_in+'train_clean.csv')
test = pd.read_csv(data_path_in+'test_clean.csv')
print(train_df.shape)
print(test_df.shape)

(523021, 35)
(45689, 33)


# Combining test & train set

In [43]:
# Dropping the 2 features not present in test set
train_stripped = train_df.copy()
train_stripped=train_stripped.drop(columns=['NumberOfCustomers','NumberOfSales'],axis=1)
all_data = pd.concat([train_stripped,test_df],axis=0).reset_index()

#to restore the original column order
all_data= all_data[list(test_df.columns.values)]
all_data.shape

(568710, 33)

# Data Visualisation

In [50]:
store_names=all_data['StoreType'].unique()
store_num = len(all_data['StoreID'].unique())
#maybe not necessary
store_type_dict = {}
store_type_count=[]
for name in store_names:
    filtered=all_data[all_data['StoreType']== name ]
    store_type_dict[name]=len(filtered['StoreID'].unique())
    store_type_count.append(len(filtered['StoreID'].unique()))
    
trace = go.Pie(labels=store_names, values=store_type_count)

py.iplot([trace], filename='basic_pie_chart')

['55.67% of total stores', '14.02% of total stores', '29.11% of total stores', '1.20% of total stores']


In [47]:
store_regions=all_data.groupby(['Region'])['StoreID'].unique()
store_dict = {}
for i in range(0,11):
    store_dict[i]=len(store_regions[i])


regional_store_types ={}
for region in range(0,11):
    region_df = all_data[all_data['Region']==region]
    reg_store_names=region_df['StoreType'].unique()
    reg_store_num = len(region_df['StoreID'].unique())
    store_type_dict = {}
    
    for name in reg_store_names:
        store_filter=region_df[region_df['StoreType']== name ]
        store_type_dict[name]=len(store_filter['StoreID'].unique())
    
    # If a store type is not present in the region create a key with value 0
    for name in store_names:
        if name not in reg_store_names:
            store_type_dict[name]=0
    
    for k,i in store_type_dict.items():
        percentage = (i/reg_store_num)*100
    regional_store_types[region] = store_type_dict
      
hyperm = []
superm = []
standm = []
shopp  = []

plot_dict ={}
for i,k in regional_store_types.items():
    hyperm.append(k["Hyper Market"])
    superm.append(k["Super Market"])
    standm.append(k["Standard Market"])
    shopp.append(k["Shopping Center"])

hyperm_bar = go.Bar(
    x=np.arange(0,12),
    y=hyperm,
    name=store_names[0]
)
superm_bar = go.Bar(
    x=np.arange(0,12),
    y=superm,
    name=store_names[1]
)

standm_bar = go.Bar(
    x=np.arange(0,12),
    y=standm,
    name=store_names[2]
)

shopp_bar = go.Bar(
    x=np.arange(0,12),
    y=shopp,
    name=store_names[3]
)


data = [hyperm_bar,superm_bar,standm_bar,shopp_bar]
layout = go.Layout(
    barmode='stack',
    title='Store types count per Region',
    xaxis=dict(title='Region'),
    yaxis=dict(title='Store')
    )

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-bar')